In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Question 1: Top 10 Bundesliga teams over entire period

**Metric: Points Per Game (PPG)**
- 3 points for win
- 1 point for draw
- 0 points for loss

Same point system as used in real life, ensures fairness, eliminates goal outliers and also gives points for draws. 

In [ ]:
# Load data
df = pd.read_csv('data/bundesliga.csv')

# Convert Date to datetime 
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)

# Quick look
print(f"Dataset shape: {df.shape}")
print(f"\nDate range: {df['Date'].min()} to {df['Date'].max()}")
df.head()

In [ ]:
# Check for missing values
df.info()

## Calculate Points for Each Team

In [ ]:
def calculate_team_stats(df):
    stats = {}
    
    # Process matches
    for _, row in df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        result = row['FTR']
        
        # Initialize teams if not seen before
        if home_team not in stats:
            stats[home_team] = {'games': 0, 'points': 0}
        if away_team not in stats:
            stats[away_team] = {'games': 0, 'points': 0}
        
        # Update home team stats
        stats[home_team]['games'] += 1
        
        if result == 'H': 
            stats[home_team]['points'] += 3
        elif result == 'D':     
            stats[home_team]['points'] += 1
        
        # Update away team stats
        stats[away_team]['games'] += 1
        
        if result == 'A': 
            stats[away_team]['points'] += 3
        elif result == 'D':     
            stats[away_team]['points'] += 1
    
    return stats

In [ ]:
# Calculate stats
team_stats = calculate_team_stats(df)

# Convert to df
stats_df = pd.DataFrame.from_dict(team_stats, orient='index')
stats_df.index.name = 'Team'
stats_df.reset_index(inplace=True)

# Calculate derived metrics
stats_df['PPG'] = stats_df['points'] / stats_df['games']

print(f"Total teams in dataset: {len(stats_df)}")
stats_df.head(10)

In [ ]:
# Filter teams with min 100 games and get top 10
min_games = 100

filtered_stats = stats_df[stats_df['games'] >= min_games]
top_10 = filtered_stats.sort_values('PPG', ascending=False).head(10)

print(f"=== TOP 10 BUNDESLIGA TEAMS (1993-2018) ===")
print(f"Filtered by minimum {min_games} games | Teams remaining: {len(filtered_stats)}\n")

top_10[['Team', 'games', 'PPG']]

In [ ]:
# Bar chart of top 10 teams
fig, ax = plt.subplots(figsize=(12, 8))

# PPG comparison
ax.barh(range(len(top_10)), top_10['PPG'].values, color='steelblue')
ax.set_yticks(range(len(top_10)))
ax.set_yticklabels(top_10['Team'].values)
ax.set_xlabel('Points Per Game', fontsize=12)
ax.set_title('Top 10 Bundesliga Teams by PPG (1993-2018)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, v in enumerate(top_10['PPG'].values):
    ax.text(v + 0.02, i, f'{v:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()